In [131]:
import pandas as pd
from functools import reduce

from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC, SVR

In [132]:
feature_dir = 'data/features/'

In [133]:
def run_experiment(model, X, y):
    scores = []
    kf = KFold(n_splits=5)
    for train_index, test_index in kf.split(X):
        X_train, X_test, y_train, y_test = X.iloc[train_index],X.iloc[test_index], y.iloc[train_index], y.iloc[test_index]
        model.fit(X_train, y_train)
        scores.append(model.score(X_test, y_test))
    print('Scores',scores)
    print('Average score', sum(scores)/len(scores))

In [134]:
df_twitter = pd.read_csv(feature_dir+'twitter_features.csv')
df_news = pd.read_csv(feature_dir+'news_features.csv')
df_out= pd.read_csv(feature_dir+'stock_features.csv')

In [135]:
clf = SVC(C=1.0)
clf2 = SVR(C=1.0)

In [136]:
df_y = df_out
df_X = pd.merge(df_twitter, df_news, on='Date', how='outer')

In [137]:
df_y

,Date,Change VIX,Lag 2 Change VIX,Lag 2 Significant VIX,Change SPY,Lag 2 Change SPY,Lag 2 Significant SPY
0,2015-01-02,0.116279,0.144707,1,-0.007741,-0.020787,-1
1,2015-01-05,0.080518,0.050026,1,-0.010708,-0.013469,-1
2,2015-01-06,0.059406,-0.118052,-1,-0.010188,0.009501,0
3,2015-01-07,-0.008854,-0.184119,-1,-0.003315,0.024724,1
4,2015-01-08,-0.110174,0.005020,0,0.012859,0.001961,0
...,...,...,...,...,...,...,...
499,2016-12-23,0.005300,0.044815,1,-0.000754,0.005057,0
500,2016-12-27,0.077329,0.072594,1,0.002617,-0.006814,0
501,2016-12-28,-0.030179,0.110177,1,0.002433,-0.008121,0
502,2016-12-29,0.105971,0.069962,1,-0.009225,0.002495,0


In [138]:
X = df_X.reset_index(drop=True)
y = df_y.reset_index(drop=True)
X = X.drop(columns=['Date'])
X.fillna(X.mean(), inplace=True)
y = y['Lag 2 Significant VIX']
y.fillna(y.mean(), inplace=True)

In [139]:
X

,BarackObama_sentiment_score,BarackObama_heuristic_score,cnnbrk_sentiment_score,cnnbrk_heuristic_score,KimKardashian_sentiment_score,KimKardashian_heuristic_score,New York Times_sentiment_score,CNN_sentiment_score,Washington Post_sentiment_score
0,0.816094,6.402818,-2.54383,-15.147706,1.504453,12.688323,-1.230945,-4.2102,-1.884661
1,1.220500,9.206268,-2.54383,-15.147706,1.504453,12.688323,-1.230945,-1.4093,-1.884661
2,0.643600,7.880455,-2.54383,-15.147706,4.065300,37.652221,-1.230945,1.0081,-1.884661
3,5.633600,40.112804,-2.54383,-15.147706,0.915400,8.877281,-1.230945,-0.9171,-1.884661
4,1.112100,9.476567,-2.54383,-15.147706,1.715000,16.294419,-1.230945,-2.5206,-1.884661
...,...,...,...,...,...,...,...,...,...
499,0.816094,6.402818,-3.50000,-22.894771,1.504453,12.688323,0.366700,-3.5821,-2.920500
500,0.816094,6.402818,-1.11410,-7.601923,1.504453,12.688323,-0.991300,-3.0340,-0.117200
501,0.816094,6.402818,-1.76330,-11.564171,1.504453,12.688323,2.958000,-1.1407,-1.727400
502,0.816094,6.402818,-0.24280,-1.812420,1.504453,12.688323,0.990100,-0.4297,-0.282200


In [140]:
y.value_counts()

-1    238
 1    219
 0     47
Name: Lag 2 Significant VIX, dtype: int64

In [141]:
# Guessing Most common class always
succ = y.value_counts().max()/(y.count())

In [142]:
succ

0.4722222222222222

In [143]:
run_experiment(clf, X, y)

Scores [0.45544554455445546, 0.5148514851485149, 0.48514851485148514, 0.49504950495049505, 0.52]
Average score 0.4940990099009901


In [144]:
def rfc_param_selection(X, y):
    n_estims= [200, 700]
    max_feats= ['auto', 'sqrt', 'log2']
    param_grid = {'n_estimators': n_estims, 'max_features' : max_feats}
    grid_search = GridSearchCV(RandomForestClassifier(), param_grid, cv=5, verbose=1)
    grid_search.fit(X, y)
    grid_search.best_params_
    print(grid_search.best_estimator_)
    print(grid_search.best_score_)
    return grid_search.best_params_

In [147]:
def svc_param_selection(X, y):
    kernels = ['rbf','linear']
    Cs = [0.001, 0.01, 0.1, 1, 10]
    gammas = [0.001, 0.01, 0.1, 1]
    param_grid = {'C': Cs, 'gamma' : gammas, 'kernel':kernels}
    grid_search = GridSearchCV(SVC(), param_grid, cv=5, verbose=5)
    grid_search.fit(X, y)
    grid_search.best_params_
    print(grid_search.best_estimator_)
    print(grid_search.best_score_)
    return grid_search.best_params_

In [148]:
svc_param_selection(X,y)

Fitting 5 folds for each of 40 candidates, totalling 200 fits
[CV] C=0.001, gamma=0.001, kernel=rbf ................................
[CV] .... C=0.001, gamma=0.001, kernel=rbf, score=0.475, total=   0.0s
[CV] C=0.001, gamma=0.001, kernel=rbf ................................
[CV] .... C=0.001, gamma=0.001, kernel=rbf, score=0.475, total=   0.0s
[CV] C=0.001, gamma=0.001, kernel=rbf ................................
[CV] .... C=0.001, gamma=0.001, kernel=rbf, score=0.465, total=   0.0s
[CV] C=0.001, gamma=0.001, kernel=rbf ................................
[CV] .... C=0.001, gamma=0.001, kernel=rbf, score=0.465, total=   0.0s
[CV] C=0.001, gamma=0.001, kernel=rbf ................................
[CV] .... C=0.001, gamma=0.001, kernel=rbf, score=0.480, total=   0.0s
[CV] C=0.001, gamma=0.001, kernel=linear .............................
[CV] . C=0.001, gamma=0.001, kernel=linear, score=0.436, total=   0.0s
[CV] C=0.001, gamma=0.001, kernel=linear .............................
[CV] . C=0.001,

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s


[CV] ...... C=0.001, gamma=0.1, kernel=rbf, score=0.475, total=   0.0s
[CV] C=0.001, gamma=0.1, kernel=rbf ..................................
[CV] ...... C=0.001, gamma=0.1, kernel=rbf, score=0.465, total=   0.0s
[CV] C=0.001, gamma=0.1, kernel=rbf ..................................
[CV] ...... C=0.001, gamma=0.1, kernel=rbf, score=0.465, total=   0.0s
[CV] C=0.001, gamma=0.1, kernel=rbf ..................................
[CV] ...... C=0.001, gamma=0.1, kernel=rbf, score=0.480, total=   0.0s
[CV] C=0.001, gamma=0.1, kernel=linear ...............................
[CV] ... C=0.001, gamma=0.1, kernel=linear, score=0.436, total=   0.0s
[CV] C=0.001, gamma=0.1, kernel=linear ...............................
[CV] ... C=0.001, gamma=0.1, kernel=linear, score=0.505, total=   0.0s
[CV] C=0.001, gamma=0.1, kernel=linear ...............................
[CV] ... C=0.001, gamma=0.1, kernel=linear, score=0.475, total=   0.0s
[CV] C=0.001, gamma=0.1, kernel=linear ...............................
[CV] .

[CV] ....... C=0.1, gamma=0.01, kernel=rbf, score=0.475, total=   0.0s
[CV] C=0.1, gamma=0.01, kernel=rbf ...................................
[CV] ....... C=0.1, gamma=0.01, kernel=rbf, score=0.475, total=   0.0s
[CV] C=0.1, gamma=0.01, kernel=rbf ...................................
[CV] ....... C=0.1, gamma=0.01, kernel=rbf, score=0.475, total=   0.0s
[CV] C=0.1, gamma=0.01, kernel=rbf ...................................
[CV] ....... C=0.1, gamma=0.01, kernel=rbf, score=0.530, total=   0.0s
[CV] C=0.1, gamma=0.01, kernel=linear ................................
[CV] .... C=0.1, gamma=0.01, kernel=linear, score=0.455, total=   0.0s
[CV] C=0.1, gamma=0.01, kernel=linear ................................
[CV] .... C=0.1, gamma=0.01, kernel=linear, score=0.554, total=   0.0s
[CV] C=0.1, gamma=0.01, kernel=linear ................................
[CV] .... C=0.1, gamma=0.01, kernel=linear, score=0.525, total=   0.0s
[CV] C=0.1, gamma=0.01, kernel=linear ................................
[CV] .

[CV] ....... C=1, gamma=0.1, kernel=linear, score=0.530, total=   0.2s
[CV] C=1, gamma=1, kernel=rbf ........................................
[CV] ............ C=1, gamma=1, kernel=rbf, score=0.535, total=   0.0s
[CV] C=1, gamma=1, kernel=rbf ........................................
[CV] ............ C=1, gamma=1, kernel=rbf, score=0.505, total=   0.0s
[CV] C=1, gamma=1, kernel=rbf ........................................
[CV] ............ C=1, gamma=1, kernel=rbf, score=0.485, total=   0.0s
[CV] C=1, gamma=1, kernel=rbf ........................................
[CV] ............ C=1, gamma=1, kernel=rbf, score=0.455, total=   0.0s
[CV] C=1, gamma=1, kernel=rbf ........................................
[CV] ............ C=1, gamma=1, kernel=rbf, score=0.480, total=   0.0s
[CV] C=1, gamma=1, kernel=linear .....................................
[CV] ......... C=1, gamma=1, kernel=linear, score=0.455, total=   0.0s
[CV] C=1, gamma=1, kernel=linear .....................................
[CV] .

[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:  5.6min finished


{'C': 0.1, 'gamma': 0.001, 'kernel': 'linear'}

In [146]:
rfc_param_selection(X,y)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:   21.9s finished


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=700,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)
0.4861584158415841


{'max_features': 'auto', 'n_estimators': 700}

In [112]:
model = RandomForestClassifier(max_features='log2',n_estimators=200)
run_experiment(model, X, y)

Scores [0.48514851485148514, 0.5148514851485149, 0.5346534653465347, 0.5148514851485149, 0.51]
Average score 0.5119009900990099


In [160]:
model = SVC(C=1, gamma=.1, kernel='rbf')
run_experiment(model, X, y)

Scores [0.5148514851485149, 0.5445544554455446, 0.5841584158415841, 0.4158415841584158, 0.48]
Average score 0.5078811881188119
